In [78]:
import requests
import scrapy
from scrapy import Selector
import pandas as pd
import re
from bs4 import BeautifulSoup as bs
from itertools import compress
import time
import os

# European Commission communications - REGDOC

https://ec.europa.eu/search/?queryText=artificial+intelligence&query_source=REGDOC&page=1

In [69]:
def get_pdfs(url, query, source, page, f_format, locator):
    request_url = url.format(query = query, source = source, page = page, f_format = f_format)
    
    i = 5
    
    while i > 0:
        try:
            response = requests.get(request_url, timeout = 5.0)
            search_sel = Selector(text = response.content)
    
            url_list = search_sel.css(locator).extract()
            pdf_list = list(set(compress(url_list, [url.lower().endswith(".pdf") for url in url_list])))
        
            if len(pdf_list) > 0:
                return(pdf_list)
            else:
                i = i - 1
                time.sleep(0.2)
        except:
            i = i - 1
            time.sleep(0.2)
            
    return(pdf_list)

In [70]:
MAIN_URL = "https://ec.europa.eu/search/?queryText={query}&query_source={source}&swlang=en&page={page}&more_options_f_formats={f_format}"

f_format = "pdf"
query_source = "REGDOC"
query_text = "artificial+intelligence"

item_locator = "a.ecl-link::attr(href)"

pdf_list_all = []
pagenum = 1

pdf_list = get_pdfs(MAIN_URL, query_text, query_source, str(pagenum), f_format, item_locator)
pdf_list_all = pdf_list_all + pdf_list

while len(pdf_list) > 0:
    pagenum = pagenum + 1
    pdf_list = get_pdfs(MAIN_URL, query_text, query_source, str(pagenum), f_format, item_locator)
    pdf_list_all = pdf_list_all + pdf_list
    time.sleep(0.5)

In [85]:
eudir = "../data_raw/EU/"
datadir = "../data_raw/EU/regdoc/"
if not os.path.isdir(eudir):
    os.mkdir(eudir)
    
if not os.path.isdir(datadir):
    os.mkdir(datadir)

for c, pdf_url in enumerate(pdf_list_all):
    r = requests.get(pdf_url, stream = True)
    filename = re.search(".*\/(.*\.pdf)", pdf_url, re.IGNORECASE).group(1)
    with open(datadir + filename, 'wb') as f:
        f.write(r.content)
    print('Completed: {:.2f}%'.format(100.0*c/(len(pdf_list_all)-1)), end='\r')
    time.sleep(0.2)

# Blog posts
https://ec.europa.eu/digital-single-market/en/blogs/76174/76175?page=

(starts at page 0)

# Policy spider
https://ec.europa.eu/digital-single-market/en/artificial-intelligence